# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
list_of_dbs = mongo.list_database_names()
list_of_dbs

['admin',
 'autosaurus',
 'classDB',
 'config',
 'epa',
 'fruits_db',
 'local',
 'met',
 'petsitly_marketing',
 'test',
 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
list_of_collections = db.list_collection_names()
list_of_collections

['establishments']

In [7]:
# review a document in the establishments collection
one_doc = db.establishments.find_one()
one_doc

{'_id': ObjectId('650a42e0de60f6379484c4bf'),
 'FHRSID': 254719,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT15 6DY',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2022-03-24T00:00:00',
 'LocalAuthorityCode': '182',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'scores': {'Hygiene': 0, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.387974', 'latitude': '51.152225'},
 'RightToReply': '',
 'Distance': 4587.347174863443,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,
  'returncode

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "AddressLine2": "Greenwich", 
    "SchemeType": "FHRS", 
    "NewRatingPending": True 
}

In [10]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant)
result.inserted_id

ObjectId('650a43ecc282be62418b2081')

In [11]:
# Check that the new restaurant was inserted
inserted_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
inserted_restaurant

{'_id': ObjectId('650a43ecc282be62418b2081'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'AddressLine2': 'Greenwich',
 'SchemeType': 'FHRS',
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [12]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_data = establishments.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}  # 1 means include the field, 0 means exclude
)
business_type_data

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [13]:
# Update the new restaurant with the correct BusinessTypeID
filter_criteria = {"BusinessName": "Penang Flavours"}
update_operation = {"$set": {"BusinessTypeID": 1}}

update_result = establishments.update_one(filter_criteria, update_operation)
update_result.modified_count

1

In [14]:
# Confirm that the new restaurant was updated
updated_restaurant = establishments.find_one({"BusinessName": "Penang Flavours"})
updated_restaurant

{'_id': ObjectId('650a43ecc282be62418b2081'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'AddressLine2': 'Greenwich',
 'SchemeType': 'FHRS',
 'NewRatingPending': True,
 'BusinessTypeID': 1}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [15]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
dover_count

994

In [16]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})
delete_result.deleted_count

994

In [17]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})

if remaining_dover_count == 0:
    print("No establishments")
else:
    print(f"There are {remaining_dover_count} establishments")

No establishments


In [18]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()

if remaining_document:
    pprint(remaining_document)
else:
    print("No documents")

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650a42e0de60f6379484c7a6'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 'self'}],
 'meta': {'dataSource': 

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [19]:
# Change the data type from String to Decimal for longitude and latitude
from bson.decimal128 import Decimal128

pipeline = [
    {
        '$set': {
            'geocode.latitude': {
                '$convert': {
                    'input': '$geocode.latitude',
                    'to': 'decimal',
                    'onError': '$geocode.latitude'
                }
            },
            'geocode.longitude': {
                '$convert': {
                    'input': '$geocode.longitude',
                    'to': 'decimal',
                    'onError': '$geocode.longitude'
                }
            }
        }
    }
]

update_result = establishments.update_many({}, pipeline)
update_result.modified_count

38786

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
# Change the data type from String to Integer for RatingValue
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
update_non_ratings = establishments.update_many(
    {"RatingValue": {"$in": non_ratings}},
    {'$set': {"RatingValue": None}}
)
update_non_ratings.modified_count

pipeline = [
    {
        '$set': {
            'RatingValue': {
                '$convert': {
                    'input': '$RatingValue',
                    'to': 'int',
                    'onError': '$RatingValue'
                }
            }
        }
    }
]

update_result = establishments.update_many({}, pipeline)
update_result.modified_count

34695

In [21]:
# Check that the coordinates and rating value are now numbers
sample_doc = establishments.find().limit(2)
for doc in sample_doc:
    pprint(doc)

{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650a42e0de60f6379484c7a6'),
 'geocode': {'latitude': Decimal128('51.083812'),
             'longitude': Decimal128('1.195625')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel':